In [172]:
import pandas as pd
import lxml.html as lh
import io
import requests
from bs4 import BeautifulSoup
import re
import itertools
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import json
import os

# Function for transcript format 1
## June - Sept

In [196]:
def scrapeTranscriptFormat1(url):
    html = requests.get(url)
    html = html.text
    bs = BeautifulSoup(html, "lxml")
    paragraphs = bs.findAll("p")

    # remove 'u' tags embedded within each paragraph tag
    for paragraph in paragraphs:
        try:
            paragraph.find('u').decompose()
        except:
            continue

    # Retrieve speaker and speech from each paragraph tag
    speaker = []
    speech = []
    pattern = r'\[.*?\]'

    for paragraph in paragraphs:
        try:
            speechText = paragraph.text.replace(u'\xa0', u'') # remove space
            speechText = re.sub(pattern, '', speechText) # remove brackets and their contents
            speaker.append(re.search("^(.*?):", speechText).group(1)) # search for speaker name, append to list
            speech.append(re.search("^(.*):(.*)", speechText).group(2).strip(" ")) # search for speaker's speech, append to list
        except:
            #print(paragraph.text)
            continue

    # Convert to dataframe
    return pd.DataFrame({'name': speaker, 'speech': speech})

# Function transcript format 2
## Oct - Dec

In [197]:
def scrapeTranscriptFormat2(url):
    html = requests.get(url)
    html = html.text
    bs = BeautifulSoup(html, "lxml")
    paragraphs = bs.findAll("p")

    # remove 'u' tags embedded within each paragraph tag
    for paragraph in paragraphs:
        try:
            paragraph.find('a').decompose()
        except:
            continue

    # Retrieve speaker and speech from each paragraph tag
    speaker = []
    speech = []
    pattern = r'\[.*?\]'

    for paragraph in paragraphs:
        try:
            speechText = paragraph.text.replace(u'()', u'') # remove space
            speechText = re.sub(pattern, '', speechText) # remove brackets and their contents
            speaker.append(re.search("^(.*?):", speechText).group(1)) # search for speaker name, append to list
            speech.append(re.search("^(.*):(.*)", speechText).group(2).strip(" ")) # search for speaker's speech, append to list
        except:
            #print(paragraph.text)
            continue

    # Convert to dataframe
    return pd.DataFrame({'name': speaker, 'speech': speech})

# Scrape 'em

In [ ]:
june1 = scrapeTranscriptFormat1("https://www.rev.com/blog/transcript-from-first-night-of-democratic-debates")
june2 = scrapeTranscriptFormat1("https://www.rev.com/blog/transcript-from-night-2-of-the-2019-democratic-debates")

july1 = scrapeTranscriptFormat1("https://www.rev.com/blog/transcript-of-july-democratic-debate-night-1-full-transcript-july-30-2019")
july2 = scrapeTranscriptFormat1("https://www.rev.com/blog/transcript-of-july-democratic-debate-2nd-round-night-2-full-transcript-july-31-2019")

sept = scrapeTranscriptFormat1("https://www.rev.com/blog/democratic-debate-transcript-houston-september-12-2019")

october = scrapeTranscriptFormat2("https://www.rev.com/blog/october-democratic-debate-transcript-4th-debate-from-ohio")

nov = scrapeTranscriptFormat2("https://www.rev.com/blog/november-democratic-debate-transcript-atlanta-debate-transcript")

dec = scrapeTranscriptFormat2("https://www.rev.com/blog/december-democratic-debate-transcript-sixth-debate-from-los-angeles")

In [199]:
june1['month'] = 'June'
june2['month'] = 'June'

july1['month'] = 'July'
july2['month'] = 'July'

sept['month'] = 'Sept'

october['month'] = 'Oct'

nov['month'] = 'Nov'

dec['month'] = 'Dec'

In [200]:
df = pd.concat([june1, june2, july1, july2, sept, october, nov, dec])

In [202]:
df.to_csv(os.getcwd() + "\\Documents\\DemDebateTranscripts.csv",index=False)